In [ ]:
#Install the different library needed
!wget https://gist.githubusercontent.com/korakot/8409b3feec20f159d8a50b0a811d3bca/raw/63788418ef51add8a0ddd4664a97910cf674415d/draw.py
!pip install flwr
!pip install tensorflow

--2022-04-25 07:40:08--  https://gist.githubusercontent.com/korakot/8409b3feec20f159d8a50b0a811d3bca/raw/63788418ef51add8a0ddd4664a97910cf674415d/draw.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1147 (1.1K) [text/plain]
Saving to: ‘draw.py.2’

draw.py.2           100%[===================>]   1.12K  --.-KB/s    in 0s      

2022-04-25 07:40:08 (36.7 MB/s) - ‘draw.py.2’ saved [1147/1147]

  Using cached importlib_metadata-1.7.0-py2.py3-none-any.whl (31 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.11.3
    Uninstalling importlib-metadata-4.11.3:
      Successfully uninstalled importlib-metadata-4.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are

  Using cached importlib_metadata-4.11.3-py3-none-any.whl (18 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 1.7.0
    Uninstalling importlib-metadata-1.7.0:
      Successfully uninstalled importlib-metadata-1.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flwr 0.18.0 requires importlib-metadata<2.0.0,>=1.4.0; python_version < "3.8", but you have importlib-metadata 4.11.3 which is incompatible.


In [ ]:
#Import the library
import tensorflow as tf
import flwr as fl
import cv2
import numpy as np
from draw import draw
from os.path import exists
from client import MNISTClient_test,model

In [ ]:
#Create the different folder where the training images will be stored
import os
!mkdir train

for i in range(10):
  os.system(" mkdir train/" + str(i))

mkdir: cannot create directory ‘train’: File exists


In [ ]:
#Create some training example
number_to_draw = "0" #@param {type:"string"}
test_number = 0
file_name = "train/" + number_to_draw + "/test" + str(test_number) + ".jpg"

while exists(file_name):
  test_number +=1
  file_name = "train/" + str(number_to_draw) + "/test" + str(test_number) + ".jpg"

draw(filename = file_name, w=28*15, h=28*15, line_width=15)
'''28*15'''

'28*15'

In [ ]:
X_train = []
y_train = []

In [ ]:
for dirs in os.walk("train/", topdown=False):
  root, dirs1, files = dirs
  if files != []:
    for file in files:
      path = dirs[0] + "/"
      image = cv2.imread( path  + file,cv2.IMREAD_UNCHANGED)
      #make mask of where the transparent bits are
      trans_mask = image[:,:,3] == 0

      #replace areas of transparency with white and not transparent
      image[trans_mask] = [255, 255, 255, 255]

      #new image without alpha channel...
      new_img = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
      new_img = cv2.resize(new_img, dsize = (28,28), interpolation = cv2.INTER_CUBIC)
      gray = cv2.cvtColor(new_img, cv2.COLOR_BGR2GRAY)
      gray = gray
      for i in range(len(gray)):
        for j in range(len(gray[0])):
          if gray[i][j] == 255:
            gray[i][j] = 0
          else :
            gray[i][j] = 255

      cv2.imwrite("image_test1.jpg",gray)
      gray = gray/255
      gray = tf.reshape(gray,gray.shape+(1,))
      X_train.append(gray)
      y_train.append(int(path[len(path)-2]))

X_train = np.asarray(X_train)
y_train = np.asarray(y_train)


(28, 28, 1)
(28, 28, 1)
(28, 28, 1)
(28, 28, 1)
(28, 28, 1)
(28, 28, 1)
(28, 28, 1)
(28, 28, 1)
(28, 28, 1)
(28, 28, 1)
(28, 28, 1)
(28, 28, 1)
(28, 28, 1)
(28, 28, 1)
(28, 28, 1)
(28, 28, 1)
(28, 28, 1)


In [ ]:
(X_train_google, y_train_google), (X_test_google, y_test_google) = tf.keras.datasets.mnist.load_data()

In [ ]:
client = MNISTClient_test(
    X_train = X_train,
    y_train = y_train,
    X_test = X_test_google,
    y_test = y_test_google,
    model = model
)


In [ ]:
X_train.shape

(17, 28, 28, 1)

In [ ]:
y_train.dtype

dtype('int64')

In [ ]:
# Start Flower client
fl.client.start_numpy_client(server_address='90.84.246.40:8080', client=client)

INFO flower 2022-04-25 08:01:26,664 | connection.py:102 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flower 2022-04-25 08:01:26,668 | connection.py:39 | ChannelConnectivity.IDLE
DEBUG flower 2022-04-25 08:01:26,672 | connection.py:39 | ChannelConnectivity.CONNECTING
DEBUG flower 2022-04-25 08:01:26,856 | connection.py:39 | ChannelConnectivity.READY


313/313 [==============================] - 3s 11ms/step - loss: 554.9241 - accuracy: 0.2242


DEBUG flower 2022-04-25 08:02:17,978 | connection.py:121 | gRPC channel closed
INFO flower 2022-04-25 08:02:17,981 | app.py:101 | Disconnect and shut down
